#### Monday, May 27, 2024

Continuing to look into the function calling capabilities of this model.

Switched from using 'langchain3' to the newly created environment of 'mistral' which uses pytorch 2.3.0 and then installed 'pip install mistral_inference'

mamba activate mistral

#### Sunday, May 26, 2024

[mistralai/Mistral-7B-Instruct-v0.3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3) dropped 4 days ago, and supports function calling.

This code was copied from the above model link. 

mamba activate langchain3

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

#### Download

In [2]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)
mistral_models_path

PosixPath('/home/rob/mistral_models/7B-Instruct-v0.3')

In [3]:
!ls /home/rob/mistral_models/7B-Instruct-v0.3

consolidated.safetensors  params.json  tokenizer.model.v3


In [4]:
snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

'/home/rob/mistral_models/7B-Instruct-v0.3'

#### Instruct Following

In [5]:
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

In [6]:
!nvidia-smi

Mon May 27 16:32:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        Off | 00000000:01:00.0  On |                  N/A |
| 36%   58C    P0              N/A /  70W |    684MiB /  2048MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")

In [8]:
model = Transformer.from_folder(mistral_models_path)

In [9]:
!nvidia-smi

Mon May 27 16:32:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        Off | 00000000:01:00.0  On |                  N/A |
| 36%   56C    P0              N/A /  70W |    672MiB /  2048MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

In [11]:
!nvidia-smi

Mon May 27 16:32:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        Off | 00000000:01:00.0  On |                  N/A |
| 29%   55C    P0              N/A /  70W |    683MiB /  2048MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
tokens = tokenizer.encode_chat_completion(completion_request).tokens

In [13]:
!nvidia-smi

Mon May 27 16:33:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        Off | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P0              N/A /  70W |    674MiB /  2048MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)

In [15]:
!nvidia-smi

Mon May 27 16:33:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        Off | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P0              N/A /  70W |    664MiB /  2048MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [16]:
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
print(result)


Machine Learning (ML) is a subset of artificial intelligence (AI) that provides systems the ability to automatically learn and improve from experience without being explicitly programmed.

In simpler terms, Machine Learning is a way of teaching computers to learn by themselves, allowing them to make decisions or predictions based on data. This is


In [17]:
!nvidia-smi

Mon May 27 16:33:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        Off | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P0              N/A /  70W |    666MiB /  2048MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### Function Calling

In [18]:
from mistral_common.protocol.instruct.tool_calls import Function, Tool
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

In [ ]:
# tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
# model = Transformer.from_folder(mistral_models_path)

In [19]:
completion_request = ChatCompletionRequest(
    tools=[
        Tool(
            function=Function(
                name="get_current_weather",
                description="Get the current weather",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "format": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["location", "format"],
                },
            )
        )
    ],
    messages=[
        UserMessage(content="What's the weather like today in Paris?"),
        ],
)


In [20]:
tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)


[{"name": "get_current_weather", "arguments": {"location": "Paris, France", "format": "celsius"}}]


#### Generate with Transformers

If you wish to run these next few cells, then restart the kernel, else you will run out of VRAM.

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
from transformers import pipeline

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

In [3]:
!ls /home/rob/Data2/huggingface/transformers

1d959166dd7e047e57ea1b2d9b7b9669938a7e90c5e37a03961ad9f15eaea17f.fea64cd906e3766b04c92397f9ad3ff45271749cbe49829a079dd84e34c1697d.lock
models--bert-base-cased
models--bert-base-cased-finetuned-mrpc
models--bert-base-uncased
models--bert-large-uncased
models--camembert-base
models--castorini--tct_colbert-v2-hnp-msmarco
models--cognitivecomputations--dolphin-2.9-llama3-8b
models--dbmdz--bert-large-cased-finetuned-conll03-english
models--distilbert-base-cased-distilled-squad
models--distilbert-base-uncased
models--distilbert-base-uncased-finetuned-sst-2-english
models--distilgpt2
models--distilroberta-base
models--dn118--unaestheticXL
models--EleutherAI--pythia-160m
models--EleutherAI--pythia-1b
models--EleutherAI--pythia-1b-deduped
models--facebook--bart-large-cnn
models--facebook--bart-large-mnli
models--google--gemma-7b
models--google--gemma-7b-it
models--google--mt5-small
models--google--pegasus-cnn_dailymail
models--google--tapas-large-finetuned-wtq
models--google--vit-base-patch16-2

In [4]:
!ls /home/rob/.cache/huggingface/hub/

models--BAAI--bge-base-en
models--BAAI--bge-base-en-v1.5
models--bert-base-cased-finetuned-mrpc
models--bert-base-uncased
models--bert-large-uncased
models--cognitivecomputations--dolphin-2.9-llama3-8b
models--colbert-ir--colbertv2.0
models--distilbert-base-uncased-finetuned-sst-2-english
models--facebook--opt-350m
models--google--gemma-7b
models--google--gemma-7b-it
models--google-t5--t5-base
models--gpt2
models--meta-llama--Meta-Llama-3-8B
models--meta-llama--Meta-Llama-3-8B-Instruct
models--microsoft--table-transformer-structure-recognition
models--mistralai--Mistral-7B-Instruct-v0.2
models--mistralai--Mistral-7B-Instruct-v0.3
models--mistralai--Mistral-7B-v0.1
models--mixedbread-ai--mxbai-embed-large-v1
models--nomic-ai--nomic-embed-text-v1
models--NousResearch--Hermes-2-Pro-Llama-3-8B
models--nvidia--dragon-multiturn-context-encoder
models--nvidia--dragon-multiturn-query-encoder
models--nvidia--Llama3-ChatQA-1.5-8B
models--roberta-base
models--robkayinto--OrpoLlama-3-8B
models--se

In [5]:
# this will download the model again ... 
# into /home/rob/.cache/huggingface/hub/

# Neither of these next two methods work ... meh for now. 
# chatbot = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
chatbot = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3", device=device)
chatbot(messages)


/home/rob/miniforge3/envs/mistral/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 